In [25]:
import pandas as pd
import numpy as np
from collections import Counter
import gensim
import matplotlib
import matplotlib.pyplot as plt
import json 
import glob
import os,os.path
import errno

# Import the Clean JSON notebooks dataset

In [2]:
df = pd.read_json("../data/all-notebooks.json",orient="index")

# Group markdown-code pairs

In [ ]:
print(df.columns)

In [35]:
all_file_paths = glob.glob('../data/clean-notebooks-full-json/**/*.json',recursive=True)

In [34]:
print(all_file_paths[0:10])

['../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/1806927.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/2153907.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/1707222.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/2287641.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/1803957.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/2104193.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/1998847.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/2159421.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/1805310.json', '../data/clean-notebooks-full-json/favorita-grocery-sales-forecasting/1997487.json']


In [38]:

# Mkdir function
# Taken from https://stackoverflow.com/a/600612/119527
def mkdir_p(path):
    """ 
    Creates a directory at the given path, if it does not exist
    If the directory already exists, do nothing
    """
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

# Look through each raw file in /data/notebooks-full.
for file_path in all_file_paths:
    
    temp = pd.read_json(file_path)
    
    # create a new dataframe for each notebook
    df_pairs = pd.DataFrame(columns=['code', 'markdown', 'cellNum'])

    
    isMarkdownFlag = False
    markdowns = [] # list of markdown for each code-markdown pair
    
    # initialize newRow
    for row in temp.iterrows():
        index = row[0]
        rowData = row[1]
        newRow = ['', [], 0]

        
        
#         if previous cell was markdown, keep appending to markdown
        if (rowData['cell_type'] =='markdown'):

            if isMarkdownFlag == True:
                
                markdowns.append(rowData['source'])
            
            # else start a new row
            else:
                markdowns = [rowData['source']]
            isMarkdownFlag = True

                
                
            
        # else code or raw
        else:
            newRow[0] = rowData['source']  # set code
            newRow[1] = markdowns  # set code
            isMarkdownFlag = False
            # set the ending index
            newRow[2] = index
            df_pairs.loc[len(df_pairs)] = newRow
            newRow = ['', [], index]

        
    
    
    # create dir if not existing
    # creates the directory clean-notebooks-full-json/[competitionname]
    newFilePath = '../data/clean-paired-' + file_path
    dirName = '../data/clean-paired-' + os.path.dirname(file_path)
    mkdir_p(dirName)
    df_pairs.to_json(newFilePath, orient='index')